<a href="https://colab.research.google.com/github/LobovaSasha/DataScience_with_Python_libraries/blob/main/06_database_%D0%A4%D0%B0%D0%BC%D0%B8%D0%BB%D0%B8%D1%8F_%D0%98%D0%BC%D1%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Работа с базами данных

Материалы:
* Макрушин С.В. Лекция 6: Работа с базами данных
* https://sqliteonline.com/
* https://docs.python.org/3/library/sqlite3.html
* https://www.geeksforgeeks.org/sql-join-set-1-inner-left-right-and-full-joins/
* https://www.datacamp.com/community/tutorials/group-by-having-clause-sql

## Задачи для совместного разбора

1. Работая с базой данных `Chinook_Sqlite.sqlite`, найдите и выведите на экран имена и фамилии всех заказчиков из Канады

2. Найти и вывести на экран названия всех альбомов группы Accept

3. Создайте базу данных с названием вашей группы. В этой базе данных создайте таблицу Student, содержащую 2 столбца: id и name. Добавьте в таблицу Student информацию о студентах, сидящих с вами по соседству.

## Лабораторная работа 6

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import  sqlite3
sqlite3.apilevel

'2.0'

In [ ]:
sqlite3.sqlite_version

'3.31.1'

1. Создайте файл базы данных c названием `recipes.db`. Создайте объект-курсор.

In [ ]:
con = sqlite3.connect('gdrive/My Drive/Colab Notebooks/recipes.db')

In [ ]:
cur = con.cursor()

2. Напишите и выполните SQL-запрос для создания таблицы рецептов `Recipe`. Таблица должна содержать следующие поля:
`id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`. Определитесь с типами и составом ключевых полей.

In [ ]:
sql = """
CREATE TABLE Recipe
(
    id integer PRIMARY KEY,
    name text,
    minutes integer,
    submitted  text,
    description text,
    n_ingredients integer
); """

In [ ]:
try:
  cur.executescript(sql)
except sqlite3.DatabaseError as err:
  print('Error: ', err)
else:
  print('Request succeeded')

Request succeeded


3. Напишите и выполните SQL-запрос для создания таблицы отзывов `Review`. Таблица должна содержать следующие поля:
`id`, `user_id`, `recipe_id`, `date`, `rating`, `review`. Определитесь с типами полей, набором ключевых полей. При помощи внешнего ключа соедините две таблицы.

In [ ]:
sql = """
CREATE TABLE Review
(
    id integer PRIMARY KEY,
    user_id integer,
    recipe_id integer,
    date text,
    rating integer,
    review text,
    FOREIGN KEY (recipe_id) REFERENCES Recipe (id)
); """

try:
  cur.executescript(sql)
except sqlite3.DatabaseError as err:
  print('Error: ', err)
else:
  print('Request succeeded')

Request succeeded


4. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample_with_tags_ingredients.csv` (__ЛР5__) в созданные таблицы

In [ ]:
import pandas as pd

reviews_csv = pd.read_csv('gdrive/My Drive/Colab Notebooks/data/reviews_sample.csv')
recipes_csv = pd.read_csv('gdrive/My Drive/Colab Notebooks/files2/recipes_sample_with_tags_ingredients.csv')

In [ ]:
recipes_csv

In [ ]:
reviews_csv

In [ ]:
recipes_csv.to_sql('Recipe_df', con, if_exists='replace', index = False)

30000

In [ ]:
cur.execute('''SELECT * FROM Recipe_df''').fetchall()

In [ ]:
cur.execute('''
INSERT INTO Recipe (id, name, minutes, submitted, description, n_ingredients)
SELECT id, name, minutes, submitted, description, n_ingredients
FROM Recipe_df
''')

In [ ]:
# cur.execute('''SELECT * FROM Recipe''').fetchall()

In [ ]:
reviews_csv.to_sql('Review_df', con, if_exists='replace', index = False)

126696

In [ ]:
cur.execute('''
INSERT INTO Review (id, user_id, recipe_id, date, rating, review)
SELECT *
FROM Review_df
''')

In [ ]:
cur.execute('''SELECT * FROM Review''').fetchall()

5. Найдите все рецепты, для выполнения которых нужно ровно 10 ингредиентов. Выведите на экран первые 5 из найденных рецептов.

In [ ]:
cur.execute('''SELECT * FROM Recipe WHERE n_ingredients=10''').fetchmany(5)

[(176, 'cola cake', 55, '1999-08-22', None, 10),
 (246, 'lee s hot crab dip', 45, '1999-09-01', "lee's hot crab dip", 10),
 (289,
  'feijoada  brazilian bean soup  ii',
  150,
  '1999-10-04',
  'brazilian bean soup',
  10),
 (373,
  'tiramisu   balducci s in new york city',
  95,
  '1999-08-17',
  'this is an adopted recipe. i shall be trying it out and then post any additional information if needs be. if you are going to give any or no stars, please at least give the reason why so that you can help me improve the recipe. thank you.',
  10),
 (544, 'greek mushroom salad', 38, '1999-08-10', None, 10)]

6. Найдите название рецепта, для выполнения которого требуется больше всего времени.

In [ ]:
cur.execute('''SELECT name, MAX(minutes) FROM Recipe''').fetchall()

[('strawberry liqueur', 129615)]

7. Запросите у пользователя id рецепта и верните информацию об этом рецепте. Если рецепт отсуствует, выведите соответствующее сообщение.

In [ ]:
user_recipe_id = int(input())


if cur.execute('''SELECT * FROM Recipe WHERE id=?''', (user_recipe_id, )).fetchall() == []:
  print('There is no recipe with selected id.')
else:
  print(cur.execute('''SELECT * FROM Recipe WHERE id=?''', (user_recipe_id, )).fetchall())


44123
[(44123, 'george s at the cove  black bean soup', 90, '2002-10-25', "an original recipe created by chef scott meskan, george's at the cove. we enjoyed this when we visited this restaurant in la jolla, california. this recipe is requested so often, they have it printed and ready at the hostess stand. it's unbeatable at the restaurant, but i do a pretty good job at home, too, if i do say so myself!", 18)]


8. Найдите кол-во отзывов с рейтингом 5.

In [ ]:
cur.execute('''
SELECT COUNT(*) FROM Review WHERE rating=5
''').fetchall()[0][0]

91361

9. Найдите кол-во уникальных рецептов, не имеющих отзывов с рейтингом, меньше 4.

In [ ]:
cur.execute('''
SELECT COUNT(*) FROM Review WHERE review IS NULL AND rating<4
''').fetchall()[0][0]

0

In [ ]:
reviews_csv[reviews_csv['review'].isna()]

,Unnamed: 0,user_id,recipe_id,date,rating,review
8881,56957,2001567544,9054,2017-06-03,5,NaN
12129,273481,2002120930,167202,2018-04-18,4,NaN
18578,56955,2001508809,9054,2017-04-26,5,NaN
21913,783599,2001959957,335709,2018-01-28,5,NaN
33232,1089381,2002299388,15553,2018-10-13,4,NaN
42783,266359,2002060663,107786,2018-03-18,5,NaN
42855,1078013,2001346906,522925,2017-01-11,4,NaN
43551,841853,1800574364,391660,2018-08-22,5,NaN
53998,1020267,2002005298,245231,2018-02-22,5,NaN
55100,56966,2001826405,9054,2017-11-26,5,NaN


In [ ]:
len(cur.execute('''
    SELECT *
    FROM Review
    WHERE
    rating<4
    AND
    review IS NULL
''').fetchall())

0

10. Найдите кол-во рецептов, опубликованных в 2010 году и имеющих длину не менее 15 минут.

In [ ]:
len(cur.execute('''
    SELECT * FROM Recipe
    WHERE submitted<'2010-01-01'
    AND minutes>=15
''').fetchall())

22550

In [ ]:
len(recipes_csv[(recipes_csv.submitted<'2010-01-01') & (recipes_csv.minutes>=15)])

22550

11. Выберите id рецепта, название рецепта, id пользователя, оставившего отзыв, дату отзыва и рейтинг для тех рецептов, которые имеют не менее 3 ингредиентов. Отсортируйте результат по id рецепта.

In [ ]:
cur.execute('''
  SELECT Recipe.id, Recipe.name, Review.user_id, Review.date, Review.rating
  FROM Recipe
  LEFT JOIN Review
  ON Recipe.id = Review.recipe_id
  WHERE Recipe.n_ingredients>=3
  ORDER BY Recipe.id
''').fetchall()